In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from datetime import datetime as dt
import datetime

import MP_functions as dfutil

# Importing & Loading Data

Make sure to unzip the pickles.zip into the root folder for the project.

In [2]:
# from load_pickles import df as pickle
df = dfutil.open_dataframe_pickle('MTA_DATA_SPRING_2014_to_2016_FULL.pickle')
dtp = dfutil.open_dataframe_pickle('daily_throughput.pickle')
htp = dfutil.open_dataframe_pickle('hourly_throughput.pickle')

# Functions

In [3]:
def get_daily_counts(row, max_counter, cols):
    """
    cols - a list of columns to compute the difference from with format [x, y] = x-y
    """
    counter = row[cols[0]] - row[cols[1]]
    if counter < 0:
        # May be counter is reversed?
        counter = -counter
    if counter > max_counter:
#         print(row[cols[0]], row[cols[1]])
        counter = min(row[cols[0]], row[cols[1]])
    if counter > max_counter:
        # Check it again to make sure we are not giving a counter that's too big
        return 0
    return counter

In [4]:
def cleanData(frame):
    # Strip whitespace from column names
    frame.columns = [s.strip() for s in frame.columns.values]

    # Create datetime column with datetime datatype
    frame['Datetime'] = pd.to_datetime(frame.DATE + ' ' + frame.TIME,
                                        format = '%m/%d/%Y %H:%M:%S')
    frame['Day_of_week'] = frame['Datetime'].apply(lambda row: row.strftime("%A"))
    
    # Chain methods together to further clean data:
        # drop old date and time columns
        # rename columns
    
    dict_col_rename = {'C/A' : 'C_A', 'UNIT' : 'Unit', 'STATION' : 'Station', 'LINENAME' : 'Linename',
                      'DIVISION' : 'Division', 'DESC' : 'Desc', 'ENTRIES' : 'Entries', 'EXITS' : 'Exits',
                      'DATE' : 'Date'}    

    frame2 = frame.rename(columns = dict_col_rename)
    
    # Check uniqueness of rows/indexes by getting counts.
    (frame2
     .groupby(['C_A', 'Unit', 'SCP', 'Station', 'Datetime'])
     .count() #.Entries.count()
     .reset_index()
     .sort_values('Entries', ascending = False)
    )
    
    # Drop duplicates.
    return frame2.drop_duplicates(subset=['C_A', 'Unit', 'SCP', 'Station', 'Datetime'])
    
    

In [5]:
def calcDailyThroughput(frame):
    
    # group daily entries and daily exits
    daily_entries = (frame
                      .groupby(['C_A', 'Unit', 'SCP', 'Station', 'Date'])
                      .Entries
                      .first()
                      .reset_index()
                    )

    daily_exits = (frame
                    .groupby(['C_A', 'Unit', 'SCP', 'Station', 'Date'])
                    .Exits
                    .first()
                    .reset_index()
                   )
    
    # Calculate the differences by day
    daily_entries[["Prev_date", "Prev_entries"]] = (daily_entries
                                                     .groupby(["C_A", "Unit", "SCP", "Station"])["Date", "Entries"]
                                                     .transform(lambda grp: grp.shift(1)))

    daily_exits[["Prev_date", "Prev_exits"]]   = (daily_exits
                                                   .groupby(["C_A", "Unit", "SCP", "Station"])["Date", "Exits"]
                                                   .transform(lambda grp: grp.shift(1)))

    # Drop all the null values generated above
    daily_entries.dropna(subset=["Prev_date"], axis=0, inplace=True)
    daily_exits.dropna(subset=["Prev_date"], axis=0, inplace=True)


    daily_entries["Daily_Entries"] = daily_entries.apply(get_daily_counts, axis=1, args=(1000000, ['Entries', 'Prev_entries']))
    daily_exits["Daily_Exits"] = daily_exits.apply(get_daily_counts, axis=1, args=(1000000, ['Exits', 'Prev_exits']))
    
    daily = pd.merge(daily_entries, daily_exits, on=['C_A','Unit','SCP', 'Station', 'Date', 'Prev_date'])
    daily['Total_throughput'] = daily['Daily_Entries'] + daily['Daily_Exits']
    
    return (daily
             .groupby(['Station', 'Date'])
             .sum()
            #  .sort_values(by=['Total_throughput'], ascending=False)
             .loc[:,['Daily_Entries', 'Daily_Exits', 'Total_throughput']]
            #  .frame()
            )

In [6]:
def calcHourThroughput(frame):
    shift = frame.copy()

    shift[['Datetime_Prev', 'Entries_Prev', 'Exits_Prev']] = (shift
                .groupby(['C_A', 'Unit', 'SCP', 'Station'])['Datetime', 'Entries', 'Exits']
                .transform(lambda grp: grp.shift(1)))
    
    shift['Entries'] = shift['Entries'] - shift['Entries_Prev']
    shift['Exits'] = shift['Exits'] - shift['Exits_Prev']
    shift = shift.dropna(how = 'any')
    
    shift['Throughput'] = shift['Entries'] + shift['Exits']
    
    return shift.loc[:,['Station','Datetime','Throughput']]

In [ ]:
# Make note to save and open pickles here:
# dfutil.save_dataframe_as_pickle(df,'MTA_DATA_SPRING_2014_to_2016_FULL.pickle')
# df_pickle = dfutil.open_dataframe_pickle('MTA_DATA_SPRING_2014_to_2016_FULL.pickle') # returns a df

# Function Testing

In [ ]:
# To find hour through put 
# test_df3.groupby(['Station', 'Datetime']).sum()

In [7]:
df.head()

,C_A,Unit,SCP,Station,Linename,Division,Date,TIME,Desc,Entries,Exits,Datetime,Day_of_week
0,A002,R051,02-00-00,59 ST,NQR456,BMT,06/18/2016,00:00:00,REGULAR,5704958,1934814,2016-06-18 00:00:00,Saturday
1,A002,R051,02-00-00,59 ST,NQR456,BMT,06/18/2016,04:00:00,REGULAR,5704958,1934829,2016-06-18 04:00:00,Saturday
2,A002,R051,02-00-00,59 ST,NQR456,BMT,06/18/2016,08:00:00,REGULAR,5704958,1934886,2016-06-18 08:00:00,Saturday
3,A002,R051,02-00-00,59 ST,NQR456,BMT,06/18/2016,12:00:00,REGULAR,5704958,1934993,2016-06-18 12:00:00,Saturday
4,A002,R051,02-00-00,59 ST,NQR456,BMT,06/18/2016,16:00:00,REGULAR,5704958,1935069,2016-06-18 16:00:00,Saturday


In [8]:
dtp.head()

Daily_Entries  Daily_Exits  Total_throughput
Station Date                                                    
1 AVE   02/28/2015      2149122.0     341314.0         2490436.0
        02/28/2016      2153712.0     342328.0         2496040.0
        02/29/2016        14018.0      15107.0           29125.0
        03/01/2015      2151936.0     342214.0         2494150.0
        03/01/2016      2156668.0     343143.0         2499811.0

In [9]:
htp.head()

,Station,Datetime,Throughput
1,59 ST,2016-06-18 04:00:00,15.0
2,59 ST,2016-06-18 08:00:00,57.0
3,59 ST,2016-06-18 12:00:00,107.0
4,59 ST,2016-06-18 16:00:00,76.0
5,59 ST,2016-06-18 20:00:00,63.0


In [ ]:
htp.groupby(['Station', 'Datetime']).sum()

# TODO

Day of the week - 7 days 
- index'd by rank
- names of stations 
- each column is a time of the day

map of morning, afternoon, evening
     for each day of week


# Data Exploration 
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

## Data Cleaning

In [ ]:
# The column names contain unneeded whitespace.
df1.columns.values

In [ ]:
# The dataframe also lacks a timeseries.
df1.dtypes

In [ ]:
# Strip whitespace from column names
df1.columns = [s.strip() for s in df1.columns.values]

# Create datetime column with datetime datatype
df1['Datetime'] = pd.to_datetime(df1.DATE + ' ' + df1.TIME,
                                 format = '%m/%d/%Y %H:%M:%S')

df1.iloc[:5]

In [ ]:
# Chain methods together to further clean data:
    # drop old date and time columns
    # rename columns
    
dict_col_rename = {'C/A' : 'C_A', 'UNIT' : 'Unit', 'STATION' : 'Station', 'LINENAME' : 'Linename',
                  'DIVISION' : 'Division', 'DESC' : 'Desc', 'ENTRIES' : 'Entries', 'EXITS' : 'Exits',
                  'DATE' : 'Date'}    

df2 = (df1
       #.drop('DATE', axis = 1)
       .drop('TIME', axis = 1)
       .rename(columns = dict_col_rename)
      )

df2.iloc[:5]

# Removing Erroneous Observations

In [ ]:
# Check uniqueness of rows/indexes by getting counts.
(df2
 .groupby(['C_A', 'Unit', 'SCP', 'Station', 'Datetime'])
 .count() #.Entries.count()
 .reset_index()
 .sort_values('Entries', ascending = False)
 .iloc[:5]
)

In [ ]:
# On 3/25, we don't seem to have two entries for same time, but let's take a look anyway.

mask = ((df2["C_A"] == "A002") & 
(df2["Unit"] == "R051") & 
(df2["SCP"] == "02-00-00") & 
(df2["Station"] == "59 ST") &
(df2["Datetime"].dt.date == datetime.datetime(2017, 3, 25).date()))
df2[mask].head()

In [ ]:
df2.Desc.value_counts()

In [ ]:
# Questions for the future, if there is time:
    # Are there other values of DESC?
    # Are there other fields to check for odd values?

# Drop duplicates.
df_no_dupe = df2.drop_duplicates(subset=['C_A', 'Unit', 'SCP', 'Station', 'Datetime'])

# Check uniqueness again after data cleaning to confirm cleanness.

In [ ]:
df_no_dupe.head()

# Calculating the Entries and Exits per Day

In [ ]:
df_daily_entries = (df_no_dupe
            .groupby(['C_A', 'Unit', 'SCP', 'Station', 'Date'])
            .Entries
            .first()
            .reset_index()
           )

df_daily_exits = (df_no_dupe
            .groupby(['C_A', 'Unit', 'SCP', 'Station', 'Date'])
            .Exits
            .first()
            .reset_index()
           )


In [ ]:
# Calculate the differences by day
df_daily_entries[["Prev_date", "Prev_entries"]] = (df_daily_entries
                                                       .groupby(["C_A", "Unit", "SCP", "Station"])["Date", "Entries"]
                                                       .transform(lambda grp: grp.shift(1)))

df_daily_exits[["Prev_date", "Prev_exits"]]   = (df_daily_exits
                                                       .groupby(["C_A", "Unit", "SCP", "Station"])["Date", "Exits"]
                                                       .transform(lambda grp: grp.shift(1)))

# Drop all the null values generated above
df_daily_entries.dropna(subset=["Prev_date"], axis=0, inplace=True)
df_daily_exits.dropna(subset=["Prev_date"], axis=0, inplace=True)

In [ ]:
df_daily_entries.head()

In [ ]:
# Check for any counters that have been reversed
df_daily_entries[df_daily_entries["Entries"] < df_daily_entries["Prev_entries"]].head()

# WTC: Is this something that can be solved by sorting before applying the transform above?

In [ ]:
##### HAVE NOT EDITED YET

# Pick a value from one of the counters reversed above & check for it 
# What's the deal with counter being in reverse
# mask = ((turnstiles_df["C/A"] == "A011") & 
# (turnstiles_df["UNIT"] == "R080") & 
# (turnstiles_df["SCP"] == "01-00-00") & 
# (turnstiles_df["STATION"] == "57 ST-7 AV") &
# (turnstiles_df["DATE_TIME"].dt.date == datetime.datetime(2016, 8, 27).date()))
# turnstiles_df[mask].head()

In [ ]:
# Let's see how many stations have this problem

(df_daily_entries[df_daily_entries["Entries"] < df_daily_entries["Prev_entries"]]
    .groupby(["C_A", "Unit", "SCP", "Station"])
    .size())

In [ ]:
# def get_daily_counts(row, max_counter, cols):
#     counter = row[cols[0]] - row[cols[1]]
#     if counter < 0:
#         # May be counter is reversed?
#         counter = -counter
#     if counter > max_counter:
#         print(row[cols[0]], row[cols[1]])
#         counter = min(row[cols[0]], row[cols[1]])
#     if counter > max_counter:
#         # Check it again to make sure we are not giving a counter that's too big
#         return 0
#     return counter

# If counter is > 1Million, then the counter might have been reset.  
# Just set it to zero as different counters have different cycle limits
df_daily_entries["Daily_Entries"] = df_daily_entries.apply(get_daily_counts, axis=1, args=(1000000, ['Entries', 'Prev_entries']))
df_daily_exits["Daily_Exits"] = df_daily_exits.apply(get_daily_counts, axis=1, args=(1000000, ['Exits', 'Prev_exits']))

In [ ]:
df_daily_entries.head()

In [ ]:
df_daily_exits.head()

In [ ]:
df_daily = pd.merge(df_daily_entries, df_daily_exits, on=['C_A','Unit','SCP', 'Station', 'Date', 'Prev_date'])
df_daily['Total_throughput'] = df_daily['Daily_Entries'] + df_daily['Daily_Exits']
df_daily.head()

In [ ]:
(df_daily
 .groupby(['Station', 'Date'])
 .sum()
#  .sort_values(by=['Total_throughput'], ascending=False)
 .loc[:,['Daily_Entries', 'Daily_Exits', 'Total_throughput']]
#  .frame()
)

# Calculating the Entries and Exits per Hour


In [ ]:
# Use concat (as a join) to fix ("de-cumulate") the columns Entries and Exits

df_shift = (df_no_dupe
            .copy()
            .drop('Linename', axis = 1)
            .drop('Division', axis = 1)
           )

df_shift[['Datetime_Prev', 'Entries_Prev', 'Exits_Prev']] = (df_shift
            .groupby(['C_A', 'Unit', 'SCP', 'Station'])['Datetime', 'Entries', 'Exits']
            .transform(lambda grp: grp.shift(1)))

df_shift.head()


# Legacy

# df_shift.columns
# df_shift['Datetime_Prev', 'Entries_Prev', 'Exits_Prev'] = (
#     df_no_dupe#[['C_A', 'Unit', 'SCP', 'Station', 'Datetime', 'Entries', 'Exits']]
#             .groupby(['C_A', 'Unit', 'SCP', 'Station'])['Datetime', 'Entries', 'Exits']
#             #.groupby(["C/A", "UNIT", "SCP", "STATION"])["DATE", "ENTRIES"]
#             .transform(lambda grp: grp.shift(1))
#             #.shift(periods = 1)
#             #.rename(columns = {'Entries' : 'Entries_Shift', 'Exits' : 'Exits_Shift', 
#             #                   'Datetime' : 'Prev_datetime'})
#            )

In [ ]:
df_shift['Entries'] = df_shift['Entries'] - df_shift['Entries_Prev']
df_shift['Exits'] = df_shift['Exits'] - df_shift['Exits_Prev']
df_shift = df_shift.dropna(how = 'any')

df_shift.head()

In [ ]:
# Challenge 3 - Total Daily Entries
#df3or4['Datetime'].dt.date == datetime.datetime(YYYY, MM, DD).date()

In [ ]:
# def get_daily_counts(row, max_counter):
#     counter = abs(row["ENTRIES"] - row["PREV_ENTRIES"])
    
#     if counter > max_counter:
#         print(row["ENTRIES"], row["PREV_ENTRIES"])
#         return 0
#     return counter

# # If counter is > 1Million, then the counter might have been reset.  
# # Just set it to zero as different counters have different cycle limits
# _ = turnstiles_daily.apply(get_daily_counts, axis=1, max_counter=1000000)